In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/gdrive', force_remount=True)

  FOLDERNAME = 'KIROV2'
  %cd /content/gdrive/My Drive/$FOLDERNAME
except ImportError:
  pass

Mounted at /content/gdrive
/content/gdrive/My Drive/KIROV2


Parseur d'entrée

In [ ]:
import json
import os
import math

class Trains:
    def __init__(self, id_groupe, read_mytrain):
        self.groupe = id_groupe
        self.id = read_mytrain["id"]
        self.sensDepart = read_mytrain["sensDepart"]
        self.voieEnLigne = read_mytrain["voieEnLigne"]
        self.voieAQuai = read_mytrain["voieAQuai"]
        self.typeCirculation = read_mytrain["typeCirculation"]
        self.typesMateriels = read_mytrain["typesMateriels"]
        self.dateHeure = read_mytrain["dateHeure"]
        self.voieProposee = read_mytrain["voieAQuai"]
        self.itineraireId = -1


class Itineraires:
    def __init__(self, read_myitineraire):
        self.id = read_myitineraire["id"]
        self.sensDepart = read_myitineraire["sensDepart"]
        self.voieEnLigne = read_myitineraire["voieEnLigne"]
        self.voieAQuai = read_myitineraire["voieAQuai"]


class InterdictionsQuais:
    def __init__(self, read_myinterdiction):
        self.voiesAQuaiInterdites = read_myinterdiction["voiesAQuaiInterdites"]
        self.voiesEnLigne = read_myinterdiction["voiesEnLigne"]
        self.typesMateriels = read_myinterdiction["typesMateriels"]
        self.typesCirculation = read_myinterdiction["typesCirculation"]


def import_data(myFile, myPath = "C:/Users/ruhlm/Downloads/instances",):
    os.chdir(myPath)
    with open(myFile, 'r') as myfile:
        data = myfile.read()

    obj = json.loads(data)
    read_trains = obj["trains"]
    trains_list = []
    groupes_list = []
    for id_groupe in range(len(read_trains)):
        trains_groupe = []
        for index_train in range(len(read_trains[id_groupe])):

            train_i = Trains(id_groupe, read_trains[id_groupe][index_train])
            trains_list.append(train_i)
            trains_groupe.append(train_i)
        groupes_list.append(trains_groupe)

    read_itineraires = obj["itineraires"]
    itineraires_list = []
    for index_itineraire in range(len(read_itineraires)):
        itineraire_i = Itineraires(read_itineraires[index_itineraire])
        itineraires_list.append(itineraire_i)

    voiesAQuai = obj["voiesAQuai"]
    voiesAQuai.append("notAffected")
    voiesEnLigne = obj["voiesEnLigne"]
    itineraire_null = Itineraires({'id': -2, 'sensDepart': True, 'voieEnLigne': "notAffected", 'voieAQuai': "notAffected"})
    itineraires_list.append(itineraire_null)

    read_interdictions = obj["interdictionsQuais"]
    interdictionsQuais_list = []
    for index_interdictions in range(len(read_interdictions)):
        interdictions_i = InterdictionsQuais(read_interdictions[index_interdictions])
        interdictionsQuais_list.append(interdictions_i)

    contraintes_list = obj["contraintes"]

    return trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list


def constraint_1(train_t, voiesAQuai, itineraires_list):
    q0 = voiesAQuai[len(voiesAQuai)-1]
    i0 = itineraires_list[len(itineraires_list)-1].id
    return (train_t.voieAQuai == q0 and train_t.itineraireId == i0) or (
                train_t.voieAQuai == itineraires_list[train_t.itineraireId].voieAQuai and train_t.voieEnLigne == itineraires_list[
            train_t.itineraireId].voieEnLigne)


def constraint_2(train_1, trains_list):
    for train_2 in trains_list:
        if train_1.id != train_2.id and train_1.groupe == train_2.groupe:
            return (train_1.voieAQuai == train_2.voieAQuai)
    return True


def constraint_3(train, interdictionsQuais_list):
    for interdiction in interdictionsQuais_list:
        if (train.voieEnLigne in interdiction.voiesEnLigne) or ([x for x in train.typesMateriels if x in interdiction.typesMateriels] != []) or (train.typeCirculation in interdiction.typesCirculation):
            if train.voieAQuai in interdiction.voiesAQuaiInterdites:
                return False
    return True

def solution_base(trains_list):
    for mytrain in trains_list:
        mytrain.voieAQuai = "notAffected"
        mytrain.itineraireId = -2
    return trains_list

def premiere_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list):
    n = len(trains_list)
    m = len(itineraires_list)
    for i in range(n):
        print("Tchu Tchu : " + str(i*100/n) + " % ")
        train_i = trains_list[i]
        # tant qu'on on pas parcouru tous les itinéraires ou que le train n'a pas été attribué
        j = 0
        while j < m and train_i.itineraireId == -1:
            itin_j = itineraires_list[j]
            # on vérifie d'abord que l'itinéraire a le même sens et la même voie en ligne que le train
            if train_i.sensDepart == itin_j.sensDepart and train_i.voieEnLigne == itin_j.voieEnLigne:
                quai_i = itin_j.voieAQuai
                itin_i = itin_j.id
                # puis que le quai choisi est bien celui attribué aux trains du même groupe si il y a déjà eu attribution
                groupe_i = groupes_list[train_i.groupe]
                deja_quai = -1
                for train in groupe_i:
                    if train.itineraireId != -1:
                        deja_quai = train.voieAQuai
                if deja_quai == -1 or deja_quai == quai_i:
                    # puis qu'il n'y a pas d'interdiction liée au quais
                    quai_valide = True
                    for f in interdictionsQuais_list:
                        if train_i.voieEnLigne in f.voiesEnLigne or train_i.typeCirculation in f.typesCirculation or [x for x in train_i.typesMateriels if x in f.typesMateriels] != []:
                            if quai_i in f.voiesAQuaiInterdites:
                                quai_valide = False
                    if quai_valide:
                        # puis qu'il n'y a pas d'incompatibilité avec un train à qui on a déjà attribué un itinéraire
                        id_i = train_i.id
                        incomp1_i = [c for c in contraintes_list if c[0] == id_i]
                        incomp2_i = [c for c in contraintes_list if c[2] == id_i]
                        incomp_i = [incomp1_i, incomp2_i]
                        itin_valide = True
                        for k in range(2):
                            for c in incomp_i[k]:
                                id_2 = c[(1-k)*2]
                                t = trains_list[0]
                                for train in trains_list:
                                    if train.id == id_2:
                                        t = train
                                if train.itineraireId != -1:
                                    itin_valide = False
                        if itin_valide:
                            train_i.voieAQuai = quai_i
                            train_i.itineraireId = itin_i
            j += 1

def troisieme_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list):
    for train in trains_list:
        if train.itineraireId == -1:
            train.voieAQuai = "notAffected"
            train.itineraireId = -2


def parseur_sortie(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, name):
    for mytrain in trains_list:
        if not constraint_1(mytrain, voiesAQuai, itineraires_list):
            print("Error 1")
        if not constraint_2(mytrain, trains_list):
            print("Error 2")
        if not constraint_3(mytrain, interdictionsQuais_list):
            print("Error 3")
        if mytrain.itineraireId == -2:
            mytrain.itineraireId = "notAffected"
        else:
            mytrain.itineraireId = str(mytrain.itineraireId)

    # À partir de là, les listes de train contiennent les itineraires empruntés et les quais affectés
    donnees_que_je_retourne_a_l_envoyeur = {}
    for mytrain in trains_list:
        donnees_que_je_retourne_a_l_envoyeur[mytrain.id] = {"voieAQuai":mytrain.voieAQuai,"itineraire":mytrain.itineraireId}

    name = "result_" + name + ".json"
    with open("C:/Users/ruhlm/Downloads/instances/"+name, "w") as file:
        json.dump(donnees_que_je_retourne_a_l_envoyeur, file)


def faire_tourner_solus():
    print("1ere étape")
    trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list = import_data('A.json', "C:/Users/ruhlm/Downloads/instances")
    premiere_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                         contraintes_list, groupes_list)
    troisieme_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                          contraintes_list, groupes_list)
    parseur_sortie(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list,"A")

    print("2eme étape")
    trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list  = import_data(
        'NS.json', "C:/Users/ruhlm/Downloads/instances")
    premiere_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                         contraintes_list, groupes_list)
    troisieme_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                          contraintes_list, groupes_list)
    parseur_sortie(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, "NS")

    print("3eme étape")
    trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list  = import_data(
        'PE.json', "C:/Users/ruhlm/Downloads/instances")
    premiere_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                         contraintes_list, groupes_list)
    troisieme_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                          contraintes_list, groupes_list)
    parseur_sortie(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, "PE")

    print("4eme étape")
    trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list  = import_data(
        'PMP.json', "C:/Users/ruhlm/Downloads/instances")
    premiere_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                         contraintes_list, groupes_list)
    troisieme_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                          contraintes_list, groupes_list)
    parseur_sortie(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, "PMP")


if __name__ == '__main__':
    faire_tourner_solus()


FileNotFoundError: ignored

Adrien

In [ ]:
def constraint_1(train_t):
    q0 = voiesAQuai[len(voiesAQuai)-1]
    i0 = itineraires_list[len(itineraires_list)-1].id
    return (train_t.voieAQuai == q0 and train_t.itineraireId == i0) or (
                train_t.voieAQuai == itineraires_list[train_t.itineraireId].voieAQuai and train_t.voieEnLigne == itineraires_list[
            train_t.itineraireId].voieEnLigne)

def constraint_2(train_1):
    for train_2 in trains_list:
        if train_1.id != train_2.id and train_1.groupe == train_2.groupe:
            return (train_1.voieAQuai == train_2.voieAQuai)
    return True

def constraint_3(train):
    for interdiction in interdictionsQuais_list:
        if (train.voieEnLigne in interdiction.voiesEnLigne) or ([x for x in train.typesMateriels if x in interdiction.typesMateriels] != []) or (train.typeCirculation in interdiction.typesCirculation):
            if train.voieAQuai in interdiction.voiesAQuaiInterdites:
                return False
    return True



import math

def check_contraintes(train, voieQ, trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list):
    for interdiction in interdictionsQuais_list:
        if (train.voieEnLigne in interdiction.voiesEnLigne) or (
                [x for x in train.typesMateriels if x in interdiction.typesMateriels] != []) or (
                train.typeCirculation in interdiction.typesCirculation):
            if voieQ in interdiction.voiesAQuaiInterdites:
                return False
    return True


def check_groupe(train, voieQ,trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list):
    groupe = train.groupe
    for t in trains_list:
        if t.groupe == groupe:
            if t.itineraireId != -1:
                voieQ_groupe = t.voieAQuai
                break

    return (voieQ_groupe == voieQ)


def deuxieme_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list):
    # but: trouver un itinéraire et une voie à quai pour les trains qui ont posé pb (ceux avec une incompatibilité). On va sélectionner l'incompatibilité de moindre coût
    trains_incompatibles_list = []
    for train in trains_list:
        if train.itineraireId == -1:
            trains_incompatibles_list.append(train)

    for train in trains_incompatibles_list:
        incompatibilites_list = []
        for incompatibilite in contraintes_list:
            if (incompatibilite[0] == train.id) and (
                    itineraires_list[incompatibilite[1]].sensDepart == train.sensDepart) and (
                    itineraires_list[incompatibilite[1]].voieEnLigne == train.voieEnLigne) and (
                    check_contraintes(train, itineraires_list[incompatibilite[1]].voieAQuai, trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list) and check_groupe(train,
                                                                                                              itineraires_list[
                                                                                                                  incompatibilite[
                                                                                                                      1]].voieAQuai, trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list)):
                incompatibilites_list.append(incompatibilite)
            elif (incompatibilite[2] == train.id) and (
                    itineraires_list[incompatibilite[3]].sensDepart == train.sensDepart) and (
                    itineraires_list[incompatibilite[3]].voieEnLigne == train.voieEnLigne) and (
                    check_contraintes(train, itineraires_list[incompatibilite[3]].voieAQuai, trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list) and check_groupe(train,
                                                                                                              itineraires_list[
                                                                                                                  incompatibilite[
                                                                                                                      3]].voieAQuai,trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list)):
                incompatibilites_list.append(incompatibilite)

        if len(incompatibilite) == 0:
            print("erreur: pas d'incompatibilité !'")

        meilleure_incompatibilite = -1
        cout_meilleure_incompatibilite = math.inf
        trajet_meilleure_incompatibilite = -1

        for (i, incompatibilite) in enumerate(incompatibilites_list):
            if (incompatibilite[0] == train.id):
                if cout_meilleure_incompatibilite > incompatibilite[4]:
                    meilleure_incompatibilite = i
                    trajet_meilleure_incompatibilite = incompatibilite[1]
                    cout_meilleure_incompatibilite = incompatibilite[4]
            elif (incompatibilite[2] == train.id):
                if cout_meilleure_incompatibilite > incompatibilite[4]:
                    meilleure_incompatibilite = i
                    trajet_meilleure_incompatibilite = incompatibilite[3]
                    cout_meilleure_incompatibilite = incompatibilite[4]

        if meilleure_incompatibilite == -1 and len(incompatibilites_list) !=0:
            print("pas de meilleure incompatibilité, étrange...: on prend la première")
            meilleure_incompatibilite = 0
            trajet_meilleure_incompatibilite = incompatibilites_list[0][3]
            cout_meilleure_incompatibilite = incompatibilites_list[0][4]

        train.itineraireId = trajet_meilleure_incompatibilite
        train.voieAQuai = itineraires_list[trajet_meilleure_incompatibilite].voieAQuai



Romain

In [ ]:
groupes_list = []
trains_list = []
itineraires_list = []
interdictionsQuais_list = []
contraintes_list = []


def premiere_attribution():
    n = len(trains_list)
    m = len(itineraires_list)
    for i in range(n):
        train_i = trains_list[i]
        # tant qu'on on pas parcouru tous les itinéraires ou que le train n'a pas été attribué
        j = 0
        while j < m and train_i.itineraireId == -1:
            itin_j = itineraires_list[j]
            # on vérifie d'abord que l'itinéraire a le même sens et la même voie en ligne que le train
            if train_i.sensDepart == itin_j.sensDepart and train_i.voieEnLigne == itin_j.voieEnLigne:
                quai_i = itin_j.voieAQuai
                itin_i = itin_j.id
                # puis que le quai choisi est bien celui attribué aux trains du même groupe si il y a déjà eu attribution
                groupe_i = groupes_list[train_i.groupe]
                deja_quai = -1
                for train in groupe_i:
                    if train.voieAQuai != -1:
                        deja_quai = train.voieAQuai
                if deja_quai == -1 or deja_quai == quai_i:
                    # puis qu'il n'y a pas d'interdiction liée au quais
                    quai_valide = True
                    for f in interdictionsQuais_list:
                        if train_i.voieEnLigne in f.voiesEnLigne or train_i.typeCirculation in f.typesCirculation or [x for x in train_i.typesMateriels if x in f.typesMateriels] != []:
                            if quai_i in f.voiesAQuaiInterdites:
                                quai_valide = False
                    if quai_valide:
                        # puis qu'il n'y a pas d'incompatibilité avec un train à qui on a déjà attribué un itinéraire
                        id_i = train_i.id
                        incomp1_i = [c for c in contraintes_list if c[0] == id_i]
                        incomp2_i = [c for c in contraintes_list if c[2] == id_i]
                        incomp_i = [incomp1_i, incomp2_i]
                        itin_valide = True
                        for k in range(2):
                            for c in incomp_i[k]:
                                id_2 = c[(1-k)*2]
                                t = trains_list[0]
                                for train in trains_list:
                                    if train.id == id_2:
                                        t = train
                                if train.itineraireId != -1:
                                    itin_valide = False
                        if itin_valide:
                            train_i.voieAQuai = quai_i
                            train_i.itineraireId = itin_i
            j += 1

def troisieme_attribution():
    for train in trains_list:
        if train.itineraireId == -1:
            train.voieAQuai = "notAffected"
            train.itineraireId = -2

Code Final

In [ ]:
import json
import os
import math

class Trains:
    def __init__(self, id_groupe, read_mytrain):
        self.groupe = id_groupe
        self.id = read_mytrain["id"]
        self.sensDepart = read_mytrain["sensDepart"]
        self.voieEnLigne = read_mytrain["voieEnLigne"]
        self.voieAQuai = read_mytrain["voieAQuai"]
        self.typeCirculation = read_mytrain["typeCirculation"]
        self.typesMateriels = read_mytrain["typesMateriels"]
        self.dateHeure = read_mytrain["dateHeure"]
        self.voieProposee = read_mytrain["voieAQuai"]
        self.itineraireId = -1


class Itineraires:
    def __init__(self, read_myitineraire):
        self.id = read_myitineraire["id"]
        self.sensDepart = read_myitineraire["sensDepart"]
        self.voieEnLigne = read_myitineraire["voieEnLigne"]
        self.voieAQuai = read_myitineraire["voieAQuai"]


class InterdictionsQuais:
    def __init__(self, read_myinterdiction):
        self.voiesAQuaiInterdites = read_myinterdiction["voiesAQuaiInterdites"]
        self.voiesEnLigne = read_myinterdiction["voiesEnLigne"]
        self.typesMateriels = read_myinterdiction["typesMateriels"]
        self.typesCirculation = read_myinterdiction["typesCirculation"]


def import_data(myFile, myPath = "C:/Users/ruhlm/Downloads/instances",):
    os.chdir(myPath)
    with open(myFile, 'r') as myfile:
        data = myfile.read()

    obj = json.loads(data)
    read_trains = obj["trains"]
    trains_list = []
    groupes_list = []
    for id_groupe in range(len(read_trains)):
        trains_groupe = []
        for index_train in range(len(read_trains[id_groupe])):

            train_i = Trains(id_groupe, read_trains[id_groupe][index_train])
            trains_list.append(train_i)
            trains_groupe.append(train_i)
        groupes_list.append(trains_groupe)

    read_itineraires = obj["itineraires"]
    itineraires_list = []
    for index_itineraire in range(len(read_itineraires)):
        itineraire_i = Itineraires(read_itineraires[index_itineraire])
        itineraires_list.append(itineraire_i)

    voiesAQuai = obj["voiesAQuai"]
    voiesAQuai.append("notAffected")
    voiesEnLigne = obj["voiesEnLigne"]
    itineraire_null = Itineraires({'id': -2, 'sensDepart': True, 'voieEnLigne': "notAffected", 'voieAQuai': "notAffected"})
    itineraires_list.append(itineraire_null)

    read_interdictions = obj["interdictionsQuais"]
    interdictionsQuais_list = []
    for index_interdictions in range(len(read_interdictions)):
        interdictions_i = InterdictionsQuais(read_interdictions[index_interdictions])
        interdictionsQuais_list.append(interdictions_i)

    contraintes_list = obj["contraintes"]

    return trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list


def constraint_1(train_t, voiesAQuai, itineraires_list):
    q0 = voiesAQuai[len(voiesAQuai)-1]
    i0 = itineraires_list[len(itineraires_list)-1].id
    return (train_t.voieAQuai == q0 and train_t.itineraireId == i0) or (
                train_t.voieAQuai == itineraires_list[train_t.itineraireId].voieAQuai and train_t.voieEnLigne == itineraires_list[
            train_t.itineraireId].voieEnLigne)


def constraint_2(train_1, trains_list):
    for train_2 in trains_list:
        if train_1.id != train_2.id and train_1.groupe == train_2.groupe:
            return (train_1.voieAQuai == train_2.voieAQuai)
    return True


def constraint_3(train, interdictionsQuais_list):
    for interdiction in interdictionsQuais_list:
        if (train.voieEnLigne in interdiction.voiesEnLigne) or ([x for x in train.typesMateriels if x in interdiction.typesMateriels] != []) or (train.typeCirculation in interdiction.typesCirculation):
            if train.voieAQuai in interdiction.voiesAQuaiInterdites:
                return False
    return True

def solution_base(trains_list):
    for mytrain in trains_list:
        mytrain.voieAQuai = "notAffected"
        mytrain.itineraireId = -2
    return trains_list

def premiere_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list):
    n = len(trains_list)
    m = len(itineraires_list)
    for i in range(n):
        print("Tchu Tchu : " + str(i*100/n) + " % ")
        train_i = trains_list[i]
        # tant qu'on on pas parcouru tous les itinéraires ou que le train n'a pas été attribué
        j = 0
        while j < m and train_i.itineraireId == -1:
            itin_j = itineraires_list[j]
            # on vérifie d'abord que l'itinéraire a le même sens et la même voie en ligne que le train
            if train_i.sensDepart == itin_j.sensDepart and train_i.voieEnLigne == itin_j.voieEnLigne:
                quai_i = itin_j.voieAQuai
                itin_i = itin_j.id
                # puis que le quai choisi est bien celui attribué aux trains du même groupe si il y a déjà eu attribution
                groupe_i = groupes_list[train_i.groupe]
                deja_quai = -1
                for train in groupe_i:
                    if train.itineraireId != -1:
                        deja_quai = train.voieAQuai
                if deja_quai == -1 or deja_quai == quai_i:
                    # puis qu'il n'y a pas d'interdiction liée au quais
                    quai_valide = True
                    for f in interdictionsQuais_list:
                        if train_i.voieEnLigne in f.voiesEnLigne or train_i.typeCirculation in f.typesCirculation or [x for x in train_i.typesMateriels if x in f.typesMateriels] != []:
                            if quai_i in f.voiesAQuaiInterdites:
                                quai_valide = False
                    if quai_valide:
                        # puis qu'il n'y a pas d'incompatibilité avec un train à qui on a déjà attribué un itinéraire
                        id_i = train_i.id
                        incomp1_i = [c for c in contraintes_list if c[0] == id_i]
                        incomp2_i = [c for c in contraintes_list if c[2] == id_i]
                        incomp_i = [incomp1_i, incomp2_i]
                        itin_valide = True
                        for k in range(2):
                            for c in incomp_i[k]:
                                id_2 = c[(1-k)*2]
                                t = trains_list[0]
                                for train in trains_list:
                                    if train.id == id_2:
                                        t = train
                                if train.itineraireId != -1:
                                    itin_valide = False
                        if itin_valide:
                            train_i.voieAQuai = quai_i
                            train_i.itineraireId = itin_i
            j += 1

def troisieme_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list):
    for train in trains_list:
        if train.itineraireId == -1:
            train.voieAQuai = "notAffected"
            train.itineraireId = -2


def parseur_sortie(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, name):
    for mytrain in trains_list:
        if not constraint_1(mytrain, voiesAQuai, itineraires_list):
            print("Error 1")
        if not constraint_2(mytrain, trains_list):
            print("Error 2")
        if not constraint_3(mytrain, interdictionsQuais_list):
            print("Error 3")
        if mytrain.itineraireId == -2:
            mytrain.itineraireId = "notAffected"
        else:
            mytrain.itineraireId = str(mytrain.itineraireId)

    # À partir de là, les listes de train contiennent les itineraires empruntés et les quais affectés
    donnees_que_je_retourne_a_l_envoyeur = {}
    for mytrain in trains_list:
        donnees_que_je_retourne_a_l_envoyeur[mytrain.id] = {"voieAQuai":mytrain.voieAQuai,"itineraire":mytrain.itineraireId}

    name = "result_" + name + ".json"
    with open("C:/Users/ruhlm/Downloads/instances/"+name, "w") as file:
        json.dump(donnees_que_je_retourne_a_l_envoyeur, file)


def faire_tourner_solus():
    chemin_fichiers = "C:/Users/ruhlm/Downloads/instances"
    print("1ere étape")
    trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list = import_data('A.json', chemin_fichiers)
    premiere_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                         contraintes_list, groupes_list)
    troisieme_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                          contraintes_list, groupes_list)
    parseur_sortie(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list,"A")

    print("2eme étape")
    trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list  = import_data(
        'NS.json', chemin_fichiers)
    premiere_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                         contraintes_list, groupes_list)
    troisieme_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                          contraintes_list, groupes_list)
    parseur_sortie(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, "NS")

    print("3eme étape")
    trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list  = import_data(
        'PE.json', chemin_fichiers)
    premiere_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                         contraintes_list, groupes_list)
    troisieme_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                          contraintes_list, groupes_list)
    parseur_sortie(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, "PE")

    print("4eme étape")
    trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, groupes_list  = import_data(
        'PMP.json', chemin_fichiers)
    premiere_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                         contraintes_list, groupes_list)
    troisieme_attribution(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list,
                          contraintes_list, groupes_list)
    parseur_sortie(trains_list, itineraires_list, voiesAQuai, voiesEnLigne, interdictionsQuais_list, contraintes_list, "PMP")


if __name__ == '__main__':
    faire_tourner_solus()

